# Week 19: Portfolio Construction - Mean-Variance, Risk Parity, Black-Litterman

## 🎯 Learning Objectives

By the end of this week, you will understand:
- **Mean-Variance Optimization**: Markowitz portfolio theory
- **Risk Parity**: Equal risk contribution
- **Black-Litterman**: Combining views with market equilibrium
- **Practical Constraints**: Turnover, position limits

---

## Why Portfolio Construction?

Signals alone don't make money. You need to:
- Combine multiple signals/assets
- Manage risk
- Handle constraints

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("✅ Libraries loaded!")
print("📚 Week 19: Portfolio Construction")

---

## Part 1: Mean-Variance Optimization

### Markowitz Problem

$$\min_w \frac{1}{2} w^T \Sigma w$$
$$\text{s.t. } w^T \mu = \mu_{target}, \quad w^T \mathbf{1} = 1$$

### Efficient Frontier

Set of portfolios with maximum return for given risk.

### 🤔 Simple Explanation

Mean-Variance finds the best trade-off between expected return and risk. It's like choosing between a safe bet and a risky gamble.

In [ ]:
# Generate asset data
n_assets = 5
n_days = 252 * 5

# Expected returns and covariance
np.random.seed(42)
mu = np.array([0.08, 0.10, 0.12, 0.15, 0.07])  # Annual expected returns
vol = np.array([0.15, 0.20, 0.25, 0.30, 0.10])  # Annual volatility

# Correlation matrix
corr = np.array([
    [1.0, 0.5, 0.3, 0.2, 0.1],
    [0.5, 1.0, 0.4, 0.3, 0.2],
    [0.3, 0.4, 1.0, 0.5, 0.1],
    [0.2, 0.3, 0.5, 1.0, 0.1],
    [0.1, 0.2, 0.1, 0.1, 1.0]
])

# Covariance matrix
Sigma = np.outer(vol, vol) * corr

print("Asset Parameters")
print("="*50)
for i in range(n_assets):
    print(f"Asset {i+1}: Return={mu[i]:.1%}, Vol={vol[i]:.1%}")

In [ ]:
def mean_variance_optimize(mu, Sigma, target_return=None, risk_free=0.02):
    """Mean-Variance Optimization"""
    n = len(mu)
    
    def portfolio_vol(w):
        return np.sqrt(w @ Sigma @ w)
    
    def neg_sharpe(w):
        ret = w @ mu
        vol = portfolio_vol(w)
        return -(ret - risk_free) / vol
    
    # Constraints
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]  # Weights sum to 1
    
    if target_return is not None:
        constraints.append({'type': 'eq', 'fun': lambda w: w @ mu - target_return})
    
    # Bounds (long only)
    bounds = [(0, 1) for _ in range(n)]
    
    # Initial guess
    w0 = np.ones(n) / n
    
    result = minimize(neg_sharpe, w0, method='SLSQP', bounds=bounds, constraints=constraints)
    
    return result.x

# Maximum Sharpe Portfolio
w_sharpe = mean_variance_optimize(mu, Sigma)

print("Maximum Sharpe Portfolio")
print("="*50)
for i, w in enumerate(w_sharpe):
    if w > 0.01:
        print(f"Asset {i+1}: {w:.1%}")

port_ret = w_sharpe @ mu
port_vol = np.sqrt(w_sharpe @ Sigma @ w_sharpe)
sharpe = (port_ret - 0.02) / port_vol
print(f"\nPortfolio Return: {port_ret:.1%}")
print(f"Portfolio Vol: {port_vol:.1%}")
print(f"Sharpe Ratio: {sharpe:.2f}")

In [ ]:
# Efficient Frontier
target_returns = np.linspace(0.07, 0.15, 20)
frontier_vols = []
frontier_weights = []

for target in target_returns:
    def min_vol(w):
        return w @ Sigma @ w
    
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        {'type': 'eq', 'fun': lambda w, t=target: w @ mu - t}
    ]
    bounds = [(0, 1) for _ in range(n_assets)]
    
    result = minimize(min_vol, np.ones(n_assets)/n_assets, 
                     method='SLSQP', bounds=bounds, constraints=constraints)
    
    if result.success:
        frontier_vols.append(np.sqrt(result.fun))
        frontier_weights.append(result.x)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(frontier_vols, target_returns[:len(frontier_vols)], 'b-', linewidth=2, label='Efficient Frontier')
plt.scatter(vol, mu, s=100, c='red', label='Individual Assets')
plt.scatter(port_vol, port_ret, s=200, c='green', marker='*', label='Max Sharpe')

for i in range(n_assets):
    plt.annotate(f'Asset {i+1}', (vol[i], mu[i]), xytext=(5, 5), textcoords='offset points')

plt.xlabel('Volatility (Annual)')
plt.ylabel('Expected Return (Annual)')
plt.title('Efficient Frontier')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## Part 2: Risk Parity

### The Idea

Each asset contributes equal risk to portfolio:

$$RC_i = w_i \frac{(\Sigma w)_i}{w^T \Sigma w}$$

Goal: $RC_1 = RC_2 = ... = RC_n$

### 🤔 Simple Explanation

Instead of equal weights (60/40), risk parity gives equal risk. If bonds are safer, you hold more bonds to balance risk.

In [ ]:
def risk_parity(Sigma, budget=None):
    """Risk Parity Portfolio"""
    n = len(Sigma)
    if budget is None:
        budget = np.ones(n) / n  # Equal risk budget
    
    def objective(w):
        port_vol = np.sqrt(w @ Sigma @ w)
        marginal_risk = Sigma @ w / port_vol
        risk_contrib = w * marginal_risk
        target_contrib = budget * port_vol
        return np.sum((risk_contrib - target_contrib)**2)
    
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = [(0.01, 1) for _ in range(n)]
    
    result = minimize(objective, np.ones(n)/n, method='SLSQP', 
                     bounds=bounds, constraints=constraints)
    
    return result.x

# Risk Parity Portfolio
w_rp = risk_parity(Sigma)

print("Risk Parity Portfolio")
print("="*50)

# Calculate risk contributions
port_vol_rp = np.sqrt(w_rp @ Sigma @ w_rp)
marginal_risk = Sigma @ w_rp / port_vol_rp
risk_contrib = w_rp * marginal_risk

for i in range(n_assets):
    print(f"Asset {i+1}: Weight={w_rp[i]:.1%}, Risk Contrib={risk_contrib[i]/port_vol_rp:.1%}")

print(f"\nPortfolio Vol: {port_vol_rp:.1%}")

---

## Part 3: Black-Litterman

### Combines

1. **Market equilibrium** (implied returns from market cap weights)
2. **Investor views** (your alpha signals)

### Formula

$$\mu_{BL} = [(\tau\Sigma)^{-1} + P^T\Omega^{-1}P]^{-1}[(\tau\Sigma)^{-1}\Pi + P^T\Omega^{-1}Q]$$

Where:
- $\Pi$: Equilibrium returns
- $P$: View matrix
- $Q$: View returns
- $\Omega$: View uncertainty

In [ ]:
def black_litterman(Sigma, market_weights, views_P, views_Q, tau=0.05, risk_aversion=2.5):
    """Black-Litterman model"""
    
    # Equilibrium returns
    Pi = risk_aversion * Sigma @ market_weights
    
    # View uncertainty (proportional to variance)
    Omega = np.diag(np.diag(views_P @ (tau * Sigma) @ views_P.T))
    
    # Black-Litterman formula
    tau_Sigma_inv = np.linalg.inv(tau * Sigma)
    Omega_inv = np.linalg.inv(Omega)
    
    M = np.linalg.inv(tau_Sigma_inv + views_P.T @ Omega_inv @ views_P)
    mu_BL = M @ (tau_Sigma_inv @ Pi + views_P.T @ Omega_inv @ views_Q)
    
    return mu_BL

# Market weights (market cap weighted)
market_weights = np.array([0.30, 0.25, 0.20, 0.15, 0.10])

# Views: Asset 3 will outperform by 3%, Asset 4 will underperform Asset 5 by 2%
views_P = np.array([
    [0, 0, 1, 0, 0],      # View 1: Asset 3
    [0, 0, 0, 1, -1]      # View 2: Asset 4 vs Asset 5
])
views_Q = np.array([0.03, -0.02])  # Expected outperformance

# Calculate BL returns
mu_BL = black_litterman(Sigma, market_weights, views_P, views_Q)

print("Black-Litterman Returns")
print("="*50)
for i in range(n_assets):
    print(f"Asset {i+1}: Original={mu[i]:.1%}, BL={mu_BL[i]:.1%}")

---

## Part 4: Comparison

Compare portfolio construction methods.

In [ ]:
# Compare all methods
w_equal = np.ones(n_assets) / n_assets
w_mv = mean_variance_optimize(mu, Sigma)
w_mv_bl = mean_variance_optimize(mu_BL, Sigma)
w_rp = risk_parity(Sigma)

methods = ['Equal Weight', 'Mean-Var', 'MV + BL Views', 'Risk Parity']
weights_all = [w_equal, w_mv, w_mv_bl, w_rp]

print("Portfolio Comparison")
print("="*70)
print(f"{'Method':<20} {'Return':<10} {'Vol':<10} {'Sharpe':<10}")
print("-"*70)

for method, w in zip(methods, weights_all):
    ret = w @ mu
    vol_p = np.sqrt(w @ Sigma @ w)
    sharpe = (ret - 0.02) / vol_p
    print(f"{method:<20} {ret:<10.1%} {vol_p:<10.1%} {sharpe:<10.2f}")

---

## Interview Questions

### Conceptual
1. What are the problems with mean-variance optimization?
2. When would you use risk parity over mean-variance?
3. How does Black-Litterman address estimation error?

### Technical
1. How do you estimate covariance for portfolio optimization?
2. What constraints would you add in practice?
3. How do you handle turnover constraints?

### Finance-Specific
1. Why is risk parity popular in asset allocation?
2. How would you incorporate transaction costs?
3. What's the rebalancing frequency trade-off?

---

## Key Takeaways

| Method | Pros | Cons |
|--------|------|------|
| Mean-Var | Optimal given inputs | Sensitive to estimates |
| Risk Parity | Robust, diversified | Ignores returns |
| Black-Litterman | Incorporates views | Complex |